In [88]:
import pandas as pd
import numpy as np

In [63]:
users = pd.read_csv('./dataset/users.dat', sep='::', header=None, engine='python')
movies = pd.read_csv('./dataset/movies.dat', sep='::', header=None, engine='python', encoding = "ISO-8859-1")
ratings_test = pd.read_csv('./dataset/ratings_test.dat', sep='::', header=None, engine='python')
ratings_train = pd.read_csv('./dataset/ratings_train.dat', sep='::', header=None, engine='python')

In [64]:
users = users.rename(columns={
    0 : 'UserID',
    1 : 'Gender',
    2 : 'Age',
    3 : 'Occupation',
    4 : 'Zip-code'
})

In [65]:
movies = movies.rename(columns={
    0 : 'MovieID',
    1 : 'Title',
    2 : 'Genres'
})

In [66]:
ratings_test = ratings_test.rename(columns={
    0 : 'UserID',
    1 : 'MovieID',
    2 : 'Rating',
    3 : 'Timestamp'
})

In [67]:
ratings_train = ratings_train.rename(columns={
    0 : 'UserID',
    1 : 'MovieID',
    2 : 'Rating',
    3 : 'Timestamp'
})

In [9]:
user_movie_matrix = ratings_train.pivot(index = 'MovieID', columns = 'UserID', values = 'Rating')
user_movie_matrix = user_movie_matrix.T

In [10]:
user_movie_matrix_orig = user_movie_matrix

In [23]:
user_movie_matrix = (user_movie_matrix - np.nanmean(user_movie_matrix, axis=1).reshape(-1, 1)) / np.nanstd(user_movie_matrix, axis=1).reshape(-1, 1)
user_movie_matrix.fillna(0, inplace=True)

In [24]:
svd_decomposition = np.linalg.svd(user_movie_matrix.to_numpy())

In [32]:
factors_number = 20
u = svd_decomposition[0][:, :factors_number]
e = np.diag(svd_decomposition[1][:factors_number])
v = svd_decomposition[2][:factors_number, :]

In [33]:
r2 = np.matmul(u, e)
r2 = np.matmul(r2, v)
r2 = pd.DataFrame(r2)

r2.columns = user_movie_matrix_orig.columns
r2.index = user_movie_matrix_orig.index

r2 = r2 * np.nanstd(user_movie_matrix_orig, axis=1).reshape(-1, 1) + np.nanmean(user_movie_matrix_orig, axis=1).reshape(-1, 1)
r2[r2 > 5] = 5
r2[r2 < 1] = 1

rmse = 0
for i in range(ratings_test.shape[0]):
    user_id = ratings_test.iloc[i]['UserID']
    movie_id = ratings_test.iloc[i]['MovieID']
    rating = ratings_test.iloc[i]['Rating']
    rmse += (r2.loc[user_id][movie_id] - rating) ** 2
rmse = (rmse / ratings_test.shape[0]) ** 0.5
print(rmse)

rmse = 0
for i in range(ratings_train.shape[0]):
    user_id = ratings_train.iloc[i]['UserID']
    movie_id = ratings_train.iloc[i]['MovieID']
    rating = ratings_train.iloc[i]['Rating']
    rmse += (r2.loc[user_id][movie_id] - rating) ** 2
rmse = (rmse / ratings_train.shape[0]) ** 0.5
print(rmse)

0.9491200248419471
0.906437499546492


In [29]:
n = 50
sum(svd_decomposition[1][:n]) / sum(svd_decomposition[1])

0.07299213033265774

In [89]:
model = pd.read_csv('./model/model.csv', index_col='UserID')

In [65]:
model.loc[['1', '1']]

KeyError: "None of [Index(['1', '1'], dtype='object')] are in the [index]"

In [66]:
model[model['UserID'] == 1]

,UserID,1,2,3,4,5,6,7,8,9,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,1,4.283278,4.166822,4.19144,4.165258,4.171907,4.16399,4.176286,4.15714,4.153793,...,4.153347,4.157951,4.153309,4.161254,4.15901,4.177563,4.167577,4.165588,4.162431,4.161565


In [10]:
model[model['UserID'] == 1][str(1)].values[0]

4.283278318981057

In [1]:
import pandas as pd
from tqdm import tqdm

In [20]:
ratings = pd.read_csv(
    './dataset/ratings_test.dat', sep='::',
    header=None,
    engine='python',
    names=['UserID', 'MovieID', 'Rating', 'Timestamp']
)
model = pd.read_csv('./model/model.csv')

In [21]:
def calculate_rmse(data):
    data['difference'] = (data['predict'] - data['Rating']) ** 2
    rmse = (sum(data['difference']) / data.shape[0]) ** 0.5

    return rmse

predicts = []
for i in tqdm(range(ratings.shape[0])):
    predicts.append(model[model['UserID'] == ratings.iloc[i]['UserID']][str(ratings.iloc[i]['MovieID'])].values[0])
ratings['predict'] = predicts

100%|██████████| 50005/50005 [00:44<00:00, 1131.18it/s]


In [22]:
rmse = calculate_rmse(ratings)
rmse

0.9491200248419472

In [15]:
import numpy as np
import scipy.spatial.distance as ds

vector_1 = np.array([1, 1, 1, 1])
vector_2 = np.array([12, 20, 4, 3])

dis = ds.cosine(vector_1, vector_2)
dis

0.1825174741068789

In [29]:
2

2

In [40]:
movies.sort_values(by='Title').iloc[105]

MovieID                     926
Title      All About Eve (1950)
Genres                    Drama
Name: 914, dtype: object

In [45]:
movies.sort_values(by='Title').iloc[108]

MovieID                               631
Title      All Dogs Go to Heaven 2 (1996)
Genres       Animation|Children's|Musical
Name: 626, dtype: object

In [49]:
import scipy.spatial.distance as ds
predict = pd.read_csv('./model/model.csv')
movie_id = 1374
similarity = dict()
for column in predict.columns[1:]:
    similarity[column] = ds.cosine(predict[movie_id].to_numpy(), predict[column].to_numpy())

KeyError: 1374

In [20]:
[3].to_numpy()

AttributeError: 'list' object has no attribute 'to_numpy'

In [12]:
movies_dict = dict(zip(movies['MovieID'], movies['Title']))

In [21]:
model = pd.read_csv('./model/model.csv', index_col='UserID')

In [15]:
def find_vectors_similarity(first_vector, second_vector):
    return np.sum(np.abs(np.array(first_vector) - np.array(second_vector))) / len(first_vector)

def find_movies_similarity(m1, m2):
    return find_vectors_similarity(model[m1], model[m2])

In [73]:
find_movies_similarity('108', '260')

0.4153069761545505

In [79]:
np.sum(np.abs(np.array(model['2858']) - np.array(model['260'])))

3637.8540649069714

In [78]:
np.sum(np.abs(np.array(model['1196']) - np.array(model['260'])))

548.8849616799132

In [25]:
model.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '3943', '3944', '3945', '3946', '3947', '3948', '3949', '3950', '3951',
       '3952'],
      dtype='object', length=3700)

In [14]:
model.mean(axis=0).to_dict()

{'UserID': 3020.5,
 '1': 3.851346155921636,
 '2': 3.6638210549872743,
 '3': 3.6730709267843307,
 '4': 3.686659528335323,
 '5': 3.68431359882563,
 '6': 3.743870047681229,
 '7': 3.688523297307924,
 '8': 3.698461197214876,
 '9': 3.6902889162618804,
 '10': 3.70050260760699,
 '11': 3.7274995086350335,
 '12': 3.6793633505739627,
 '13': 3.701728785848335,
 '14': 3.702253727674769,
 '15': 3.6820145121919383,
 '16': 3.7236016809659622,
 '17': 3.7435542835332427,
 '18': 3.701608749417993,
 '19': 3.6471393402395957,
 '20': 3.6845949521411083,
 '21': 3.706233449049566,
 '22': 3.6916464753877376,
 '23': 3.689882845152832,
 '24': 3.669800585920585,
 '25': 3.708937088032864,
 '26': 3.70478411709477,
 '27': 3.698721877433467,
 '28': 3.7126015905396668,
 '29': 3.7317584654422107,
 '30': 3.701453784942729,
 '31': 3.6936881255663447,
 '32': 3.78767436506385,
 '33': 3.702522488812609,
 '34': 3.766613193744199,
 '35': 3.6984502834117903,
 '36': 3.743104424301017,
 '37': 3.70297542705727,
 '38': 3.698799921

In [60]:
count = 5
movies_ratings = [['1','2','3','4','5'],[5,3,4,2,5]]

movies_ids = movies_ratings[0]
ratings = movies_ratings[1]
predicted_ratings = model
predicted_ratings['similarity'] = [
    find_vectors_similarity(
        predicted_ratings.loc[i][movies_ids],
        ratings
    ) for i in predicted_ratings.index
]
predicted_ratings = predicted_ratings.sort_values(by='similarity').iloc[:5]
predicted_ratings = predicted_ratings.drop(columns=movies_ids)
predicted_ratings = predicted_ratings.drop(columns=['similarity'])
mean_ratings = predicted_ratings.mean(axis=0).to_dict()
best_movies = list(
    {
        k: v for k, v in sorted(mean_ratings.items(), key=lambda x: x[1], reverse=True)
    }.items()
)[:count]

[[item[0] for item in best_movies], [item[1] for item in best_movies]]

[['356', '3114', '2028', '1721', '318'],
 [4.9756006048381085,
  4.795837586303361,
  4.733773349338423,
  4.723267595887378,
  4.715783642138396]]

In [61]:
model['356']

UserID
1       4.492624
2       4.461854
3       3.791523
4       4.247388
5       2.214560
          ...   
6036    3.268777
6037    4.117754
6038    3.841795
6039    3.605733
6040    3.384796
Name: 356, Length: 6040, dtype: float64

In [92]:
movies.sort_values(by='Title').iloc[910]

MovieID               1167
Title      Dear God (1996)
Genres              Comedy
Name: 1151, dtype: object